##### 导入库

In [62]:
#导入库
import pyecharts
import pandas as pd
import numpy as np
from pyecharts.charts import Liquid,Grid,Tab,Bar,Pie,Page,TreeMap,\
Timeline,Line,EffectScatter,Scatter,HeatMap,PictorialBar,Radar,Polar,Funnel,WordCloud
from pyecharts.components import Image
from pyecharts.options import ComponentTitleOpts
from pyecharts.charts import Gauge
from pyecharts.globals import SymbolType
from pyecharts.globals import ThemeType
import asyncio
from aiohttp import TCPConnector, ClientSession
from pyecharts.commons.utils import JsCode
import pyecharts.options as opts
import re
import os
import time
from sklearn.decomposition import PCA


##### 选择文件

In [63]:
n='★阴性整合.csv'
# n='硬算验证.csv'
# n='硬算验证.csv'
# p='★阳性整合.csv'
# n='0913-林职-阴性-55人.csv'

# p='纯海洛因.csv'
# p='11-23楚雄.csv'
p='纯海洛因-重复.csv'
p='纯冰毒-重复.csv'
# p='纯冰毒.csv'
# p='毛冰毒.csv'
# p='1921.csv'
# p='1019.csv'

tit=(n+'+'+p).replace('.csv', '')

##### 数据处理 选择特征

In [64]:
# 数据处理 选择特征
n_fea = pd.read_csv("E:\\desktop\\data\\fea\\[特征]"+n, encoding='gbk')
p_fea = pd.read_csv("E:\\desktop\\data\\fea\\[特征]"+p, encoding='gbk')

n_name = list(n_fea['name'])
p_name = list(p_fea['name'])

np_fea = pd.concat([n_fea, p_fea], axis=0)

fea_list = [
    #         'maxvalue',
    #         'maxindex',
    # 'minvalue',
    # 'minindex',
    #         'retvalue',
    #         'retindex',
    # 'Rrange1',
    #         'F1',
    # 'Rrange2',
    #         'Rrange3',
    #         'F2',
    # 'time1',
    # 'time2',
    # 'hh',
    #         's',
    #         'sret',
    #         'sdown',
    #         'sdigu',
    # 's_sum',
    # 'qulv',
    # 'dmq',
    # 'dnp',
    # 'var',
    #         'std',
    # 'cv',
    #         'ske',
    #         'kur',
    #         'maxang',
    #         'minang',
    #         'retang',
    # 'ang_sum',
    #         'wave',
    # 'lumpiness',
    # 'stability',
    'std1st_der',
    # 'spikiness',#G
    # 'level_shift_size',
    # 'diff2y_acf1',#G

    'attr_"angle"__coeff_3',
    # 'attr_"angle"__coeff_2',
    'attr_"imag"__coeff_3',
    # 'attr_"real"__coeff_2',
    # 'attr_"imag"__coeff_6',
    # 'attr_"abs"__coeff_1',

    # 'attr_"stderr"__chunk_len_50__f_agg_"min"',
    # 'attr_"stderr"__chunk_len_10__f_agg_"mean"',
    # 'attr_"stderr"__chunk_len_5__f_agg_"mean"',
    # 'attr_"stderr"__chunk_len_5__f_agg_"max"',
    # 'attr_"stderr"__chunk_len_5__f_agg_"min"',
    # 'attr_"stderr"__chunk_len_10__f_agg_"max"',

    # 'coeff_2',

    # 'MAC',
    # 'ASOC',

]

# all
fea_list = {'attr_"imag"__coeff_3',
            'dmq',
            'attr_"stderr"__chunk_len_50__f_agg_"min"',
            'MAC',
            'ASOC',
            'coeff_2',
            'var',
            'std',
            'attr_"abs"__coeff_1',
            'stability',
            'std1st_der',
            's',
            'attr_"stderr"__chunk_len_10__f_agg_"mean"',
            'attr_"stderr"__chunk_len_5__f_agg_"mean"',
            'attr_"stderr"__chunk_len_5__f_agg_"min"',
            'attr_"stderr"__chunk_len_5__f_agg_"max"',
            }
fea_list = [
    'maxvalue',
    'maxindex',
    'minvalue',
    'minindex',
    'retvalue',
    'retindex', 
    'Rrange1', 
    'F1', 
    'Rrange2', 
    'F2', 
    'time1', 
    'time2', 
    's', 
    'hh', 
    'sret', 
    'sdown', 
    'sdigu',
      's_sum', 
      'qulv', 
      'dmq', 
      'dnp', 
      'var', 
      'std', 
      'cv', 
      'maxang', 
      'minang', 
      'retang', 
      'ang_sum', 
      'lumpiness',
      'level_shift_size',
      'stability',
      'std1st_der',
      'attr_"imag"__coeff_3',
        'attr_"angle"__coeff_3',
        'attr_"angle"__coeff_2',
        # 'attr_"angle"__coeff_6',
        'attr_"real"__coeff_2',
      ]
tab = Tab()
page = Page(layout=Page.DraggablePageLayout)


##### 数据分析

In [65]:
#差值辨别比
gap_dict={}
for fea in fea_list:
    n_col=list(n_fea[fea])
    p_col=list(p_fea[fea])

    gap_list=[]
    gap_up=0
    gap_down=0
    for a in range(len(n_col)):
        gaps=[]
        for b in range(len(p_col)):
            gap=p_col[b]-n_col[a]
            gaps.append(gap)
            if gap>0:
                gap_up=gap_up+1
            elif gap<0:
                gap_down=gap_down+1
        gap_list.append(gaps)

    gap_bili=0
    if (gap_up+gap_down)!=0:
        if gap_up/(gap_up+gap_down)>0.5:
            gap_bili=gap_up/(gap_up+gap_down)
        else:
            gap_bili=gap_down/(gap_up+gap_down)

    gap_dict[fea]=round(gap_bili*100,1)

gap_dict = dict(sorted(gap_dict.items(), key=lambda x: x[1],reverse=True))
fea_list=list(gap_dict.keys())
gap_dict = dict(sorted(gap_dict.items(), key=lambda x: x[1],reverse=False))


In [66]:
#寻找临界
crit={}
for fea in fea_list:
    n_col=list(n_fea[fea])
    p_col=list(p_fea[fea])
    np_col=n_col+p_col
    s_dict={'s':[],'value':[],'plou':[]}
    for i in range(len(np_col)):
        s=np.sum(np.array(n_col)-np_col[i])+np.sum(np.array(p_col)-np_col[i])
        s_dict['s'].append(abs(s))
        s_dict['value'].append(np_col[i])
        if np.sum(np.array(p_col)-np_col[i])>=0:
            s_dict['plou'].append('up')
        else:
            s_dict['plou'].append('down')
    minindex=s_dict['s'].index(min(s_dict['s']))
    crit[fea]=[s_dict['value'][minindex],s_dict['plou'][minindex]]


#all
# crit={
#         'attr_"imag"__coeff_3': [37.4313838267122, 'down'],
#         'attr_"angle"__coeff_3': [148.691336284887, 'up'],
#         'std1st_der': [0.232322553980408, 'down']
#     }

#'ice':
# crit={
#         'attr_"imag"__coeff_3': [44.300323034086105, 'down'],
#         'attr_"imag"__coeff_6': [-0.3725524249792875, 'down'],
#         'attr_"angle"__coeff_3': [136.06791866143828, 'up'],
#         'attr_"angle"__coeff_6': [0.4740030783229729, 'down'],
#         'minindex': [46, 'down'],
#         'attr_"stderr"__chunk_len_50__f_agg_"min"': [1.540701710244424, 'down']
#         }
#'her':

# crit={
#         'attr_"imag"__coeff_3': [41.33070266340975, 'down'],
#         'attr_"abs"__coeff_1': [152.0738635219924, 'down'],
#         'dmq': [42.74440875803502, 'down'],
#         'attr_"stderr"__chunk_len_50__f_agg_"min"': [1.4019332792052386, 'down'],
#         'attr_"stderr"__chunk_len_10__f_agg_"max"': [0.1470643184692257, 'down'],
#         'std1st_der': [0.2414781993937969, 'down'],
#         'qulv': [125.8127309258182, 'up']
#         }

#寻找阴阳聚集点
for fea in fea_list:
    n_col=list(n_fea[fea])
    p_col=list(p_fea[fea])
    if crit[fea][1]=="up":
        p_col.sort(reverse=True) #倒序
        crit[fea].append(p_col[len(p_col)*97//100]) #阴性聚集点

        n_col.sort(reverse=False) #正序
        crit[fea].append(n_col[len(n_col)*97//100]) #阳性聚集点

    else:
        p_col.sort(reverse=False) #倒序
        crit[fea].append(p_col[len(p_col)*97//100]) #阴性聚集点

        n_col.sort(reverse=True) #正序
        crit[fea].append(n_col[len(n_col)*97//100]) #阳性聚集点


In [67]:
#寻找越界
cross_dict={}
for fea in fea_list:
    n_col=list(n_fea[fea])
    p_col=list(p_fea[fea])

    for x in range(len(n_col)):
        if crit[fea][1]=='up':
            if n_col[x]-crit[fea][0]>0:
                try:
                    cross_dict[n_name[x]][0]=cross_dict[n_name[x]][0]+1
                    cross_dict[n_name[x]][1].append(fea)
                    cross_dict[n_name[x]][2].append(abs(n_col[x]-crit[fea][0]))
                except:
                    cross_dict[n_name[x]]=[1,[fea],[abs(n_col[x]-crit[fea][0])]]
        elif crit[fea][1]=='down':
            if n_col[x]-crit[fea][0]<0:
                try:
                    cross_dict[n_name[x]][0]=cross_dict[n_name[x]][0]+1
                    cross_dict[n_name[x]][1].append(fea)
                    cross_dict[n_name[x]][2].append(abs(n_col[x]-crit[fea][0]))
                except:
                    cross_dict[n_name[x]]=[1,[fea],[abs(n_col[x]-crit[fea][0])]]

    for x in range(len(p_col)):
        if crit[fea][1]=='up':
            if p_col[x]-crit[fea][0]<0:
                try:
                    cross_dict[p_name[x]][0]=cross_dict[p_name[x]][0]+1
                    cross_dict[p_name[x]][1].append(fea)
                    cross_dict[p_name[x]][2].append(abs(p_col[x]-crit[fea][0]))
                except:
                    cross_dict[p_name[x]]=[1,[fea],[abs(p_col[x]-crit[fea][0])]]
        elif crit[fea][1]=='down':
            if p_col[x]-crit[fea][0]>0:
                try:
                    cross_dict[p_name[x]][0]=cross_dict[p_name[x]][0]+1
                    cross_dict[p_name[x]][1].append(fea)
                    cross_dict[p_name[x]][2].append(abs(p_col[x]-crit[fea][0]))
                except:
                    cross_dict[p_name[x]]=[1,[fea],[abs(p_col[x]-crit[fea][0])]]

# cross_dict

In [68]:
#检出率 误判率 精确率
rate_dict={}
for fea in fea_list:
    rate_dict[fea]=[0,0,0,0,0] #[检出人数，误判人数，检出率，误判率，精确率]

for i in range(len(n_fea)):
    row=n_fea.iloc[i]
    for fea in fea_list:
        value=row[fea]
        if crit[fea][1]=='up': #误判+1
            if value>crit[fea][0]:
                rate_dict[fea][1]=rate_dict[fea][1]+1
        else:
            if value<crit[fea][0]:
                rate_dict[fea][1]=rate_dict[fea][1]+1

for i in range(len(p_fea)):
    row=p_fea.iloc[i]
    name=p_fea.iloc[i,0]
    for fea in fea_list:
        value=row[fea]
        if crit[fea][1]=='up': #检出+1
            if value>crit[fea][0]:
                rate_dict[fea][0]=rate_dict[fea][0]+1
        else:
            if value<crit[fea][0]:
                rate_dict[fea][0]=rate_dict[fea][0]+1

for fea in fea_list:
    rate_dict[fea]=[
        round(rate_dict[fea][0]*100/len(p_fea),1), #检出率
        round(rate_dict[fea][1]*100/len(n_fea),1), #误判率
        round((rate_dict[fea][0]+len(n_fea)-rate_dict[fea][1])*100/(len(n_fea)+len(p_fea)),1) #精确率
        ]

In [69]:
#求范围
interval_dict={}
for fea in fea_list:
    n_col=list(n_fea[fea])
    p_col=list(p_fea[fea])
    np_col=n_col+p_col
    np_col.sort()
    np_col=np.array_split(np_col,10,axis=0) # 10分组数

    interval_list=[]
    for i in range (len(np_col)):
        interval = [min(np_col[i]),max(np_col[i]),0,0]
        interval_list.append(interval)
    interval_dict[fea]=interval_list

def get_interval_num(fea_list,x_fea,interval_dict,ifnp):  #ifnp 2是阴 3是阳
    interval_dict=interval_dict
    for fea in fea_list:
        x_col=list(x_fea[fea])
        for x in x_col:
            for y in range(len(interval_dict[fea])):
                if x>=interval_dict[fea][y][0] and x<interval_dict[fea][y][1]:
                        interval_dict[fea][y][ifnp]=interval_dict[fea][y][ifnp]+1
    return interval_dict

interval_dict=get_interval_num(fea_list,n_fea,interval_dict,2)
interval_dict=get_interval_num(fea_list,p_fea,interval_dict,3)

new_dict={}
for fea in fea_list:
    x=[]
    y0=[]
    y1=[]
    row=interval_dict[fea]
    for inte in row:
        x_value=str(round(inte[0],4))+'-'+str(round(inte[1],4))
        x.append(x_value)
        y0.append(inte[2])
        y1.append(inte[3])
        new_dict[fea]=(x,y0,y1)

##### 画图

In [70]:
#背景
bg_line = (
    Line(init_opts=opts.InitOpts(width="2000px",
                                 height="2000px",
                                 bg_color={"type": "pattern", "image": JsCode("img"), "repeat": "repeat"},
                                 chart_id='bg_image',
                                 ))
    .add_xaxis([None])
    .add_yaxis("", [None])
    .set_global_opts(
        title_opts=opts.TitleOpts(title=tit,
                                #   subtitle=time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())),#副标题
                                #   pos_left='center',
                                  title_textstyle_opts=opts.TextStyleOpts(font_size=30, color='#51c2d5'),
                                  pos_top='2%'
                                  ),
        yaxis_opts=opts.AxisOpts(is_show=False),
        xaxis_opts=opts.AxisOpts(is_show=False))
)
bg_line.add_js_funcs(
    """
    var img = new Image(); img.src = 'https://s1.ax1x.com/2022/10/31/xotew6.png';

    """
)

page.add(bg_line)

In [71]:
#装饰图片
zb_line = (
    Line(init_opts=opts.InitOpts(
                                #  width="370px",
                                #  height="220px",
                                 bg_color={"type": "pattern", "image": JsCode("img"), "repeat": "no-repeat"},
                                 chart_id='zb_image',
                                 )
                                 )
    .add_xaxis([None])
    .add_yaxis("", [None])
    .set_global_opts(
        yaxis_opts=opts.AxisOpts(is_show=False),
        xaxis_opts=opts.AxisOpts(is_show=False))
)

zb_line.add_js_funcs(
    """
    var img = new Image(); img.src = 'https://s1.ax1x.com/2022/10/31/xoNbKs.png';

    """
)

page.add(zb_line)

In [72]:
#阴阳比例图
pie_color_n=JsCode("""
                        new echarts.graphic.LinearGradient(
                            0, 0, 0, 1,
                            [{offset: 0, color: 'rgba(0, 221, 255)'},
                                {offset: 1, color: 'rgba(77, 119, 255)'}],
                            false)
                        """)
pie_color_p=JsCode("""
            new echarts.graphic.LinearGradient(
                0, 0, 0, 1,
                [{offset: 0, color: '#ffb199'},
                    {offset: 1, color: '#ff0844'}],
                false)
            """)
np_pie = (
    Pie(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS,chart_id='np_pie',width="400px",height="150px",))
    .add(
        series_name="阴阳占比",
        data_pair=[
            ["阴性",len(n_fea)],
            ["阳性",len(p_fea)],
        ],
        center=["55%", "50%"],
        radius=[17,50],
        rosetype="radius",
        label_opts=opts.LabelOpts(font_size=14,formatter="{b}\n{c}人\n{d}%",color='#516b91') #,font_weight='bold'
    )

    .set_colors([pie_color_n, pie_color_p])
    .set_series_opts(
                    tooltip_opts=opts.TooltipOpts(is_show=False, trigger="item",),
                    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title=''),
        legend_opts=opts.LegendOpts(is_show=False),
    )
)

page.add(np_pie)

In [73]:
#辨别比排名图
differ = (
Bar(init_opts=opts.InitOpts(
    height="810px",
    width='450px',
    theme=ThemeType.WESTEROS,
    chart_id='differ',
    ))
.add_xaxis(list(gap_dict.keys()))

.add_yaxis(""
    , list(gap_dict.values())
    ,itemstyle_opts={
        "normal": {
            "color": JsCode(
                """new echarts.graphic.LinearGradient(1, 0, 0, 0, [{
            offset: 0,
            color: '#79CDCD'
        }, {
            offset: 1,
            color: '#3b73cf'
        }], false)"""
            ),
            "barBorderRadius": [9, 9, 9, 9],
        }
    }
)


.set_series_opts(
    label_opts=opts.LabelOpts(
    position="inside", #内部居中
    font_size= 10,
    color='white',
    formatter=JsCode("function (params) {return params.name + '  [ ' +params.value + '% ]';}"),
    ),
)
.set_global_opts(
    title_opts=opts.TitleOpts(title='辨别比排名 ['+str(len(fea_list))+'个特征 ]',pos_top='2%',)
    ,xaxis_opts=opts.AxisOpts(
                            is_show=False,
                            min_='50'
                             )
    ,yaxis_opts=opts.AxisOpts(
                            is_show=False,
                            axislabel_opts=opts.LabelOpts(
                            interval=0
                            ),
                            )
    ,datazoom_opts=opts.DataZoomOpts(pos_left='left',
                                     orient="vertical",
                                     range_start=100,range_end=len(gap_dict)*1.5
                                     )
    ,tooltip_opts=opts.TooltipOpts(is_show=True)
    ,legend_opts=opts.LegendOpts(selected_mode='single',pos_bottom='bottom')
) #set_global_opts的括号
.reversal_axis()
)

page.add(differ)

In [74]:
#时间线 散点图 范围
time_sd = Timeline(init_opts=opts.InitOpts(width="1900px", height="980px",chart_id='time_sd', theme=ThemeType.WALDEN))
for fea in fea_list:
    n_col=list(n_fea[fea])
    p_col=list(p_fea[fea])
    slen=max([len(n_col),len(p_col)])
#--------------------------散点图------------------------------------------------
    sd=(
        Scatter(
            init_opts=opts.InitOpts(theme=ThemeType.WESTEROS)
                                    )
        .add_xaxis([i for i in range(slen)])

        .add_yaxis(
            series_name="阴性",
            y_axis=[list(z) for z in zip(n_col,n_name)],
            symbol_size=6,
            label_opts=opts.LabelOpts(is_show=False),

        )

        .add_yaxis(
            series_name="阳性",
            y_axis=[list(z) for z in zip(p_col,p_name)],
            symbol_size=6,
            label_opts=opts.LabelOpts(is_show=False),
            itemstyle_opts=opts.ItemStyleOpts(color='#EC7063')
        )

        .set_global_opts(
            title_opts=opts.TitleOpts(title=fea,
                                    pos_left='center',
                                    pos_top='5%',
                                    title_textstyle_opts=opts.TextStyleOpts(font_size=33,font_weight='bolder',color='#516b91')
                                    ),
            xaxis_opts=opts.AxisOpts(is_show=False),
            yaxis_opts=opts.AxisOpts(is_show=False,is_scale=True),
            legend_opts=opts.LegendOpts(pos_top='11%',),
            tooltip_opts=opts.TooltipOpts(
            formatter=JsCode(
                "function (params) {return params.value[2];}"
            ),
            axis_pointer_type="cross",
            )

    )
    )

#--------------------------散点图界限------------------------------------------------
    sd.set_series_opts(
    label_opts=opts.LabelOpts(is_show=False),
    )

    crit_line = (
        Line()
        .add_xaxis([i for i in range(slen)])
        .add_yaxis("临界线",
         [crit[fea][0] for i in range(slen)],
        is_symbol_show=False,
        label_opts=opts.LabelOpts(is_show=False),
        itemstyle_opts=opts.ItemStyleOpts(color='#8E44AD'),
        linestyle_opts=opts.LineStyleOpts(width=1),
         )
        .add_yaxis("阴性聚集点",
         [crit[fea][2] for i in range(slen)],
        is_symbol_show=False,
        label_opts=opts.LabelOpts(is_show=False),
        itemstyle_opts=opts.ItemStyleOpts(color='blue'),
        linestyle_opts=opts.LineStyleOpts(width=1, type_="dashed"),
         )
        .add_yaxis("阳性聚集点",
         [crit[fea][3] for i in range(slen)],
        is_symbol_show=False,
        label_opts=opts.LabelOpts(is_show=False),
        itemstyle_opts=opts.ItemStyleOpts(color='red'),
        linestyle_opts=opts.LineStyleOpts(width=1, type_="dashed"),
         )
        .set_global_opts(
            yaxis_opts=opts.AxisOpts(is_show=False),
            xaxis_opts=opts.AxisOpts(is_show=False))
    )

    sd=sd.overlap(crit_line)

#--------------------------检出率 误判率 精确率------------------------------------------------
    #颜色范围

    bar = (
        Bar(init_opts=opts.InitOpts(theme=ThemeType.WALDEN))
        .add_xaxis(['检出率', '误判率', '精确率','辨别比'])
        .add_yaxis("", rate_dict[fea]+[gap_dict[fea]],
        itemstyle_opts={
        "normal": {
            "color": JsCode(
                """new echarts.graphic.LinearGradient(0, 1, 0, 0, [{
            offset: 0,
            color: '#B9B6E5'
        }, {
            offset: 1,
            color: '#7DE2FC'
        }], false)"""
            ),
            "barBorderRadius": [9, 9, 0, 0],
            "shadowColor": "rgb(0, 160, 201)",
        }
    }
        )#调整颜色
        .set_global_opts(yaxis_opts=opts.AxisOpts(is_show=False,)) #隐藏Y轴
        .set_series_opts(
            label_opts=opts.LabelOpts(
            font_size= 13,
            color='#9e9f9f',
            formatter="{c}%",
            ),
        )
    )

#--------------------------范围分布图------------------------------------------------
    int_line = (
        Line()
        .add_xaxis(new_dict[fea][0])
        .add_yaxis("阴性",
        new_dict[fea][1],
        is_smooth=True,
        linestyle_opts=opts.LineStyleOpts(width=0),
        is_symbol_show=False, #是否显示数字
        areastyle_opts=opts.AreaStyleOpts(
            opacity=0.3,
            color=JsCode("""
                        new echarts.graphic.LinearGradient(
                            0, 0, 0, 1,
                            [{offset: 0, color: 'rgba(0, 221, 255)'},
                                {offset: 1, color: 'rgba(77, 119, 255)'}],
                            false)
                        """)
        )
        )

        .add_yaxis("阳性",
        new_dict[fea][2],
        is_smooth=True,
        linestyle_opts=opts.LineStyleOpts(width=0),
        is_symbol_show=False, #是否显示数字
        areastyle_opts=opts.AreaStyleOpts(
            opacity=0.3,
            color=JsCode("""
                        new echarts.graphic.LinearGradient(
                            0, 0, 0, 1,
                            [{offset: 0, color: 'rgba(255, 0, 135)'},
                                {offset: 1, color: 'rgba(135, 0, 157)'}],
                            false)
                        """)
        )
        )

        .set_global_opts(
            legend_opts=opts.LegendOpts(is_show=False,),
            yaxis_opts=opts.AxisOpts(is_show=False,),
            xaxis_opts=opts.AxisOpts(
                axistick_opts=opts.AxisTickOpts(is_align_with_label=True),
#                 is_scale=False,
                is_show=False,
                boundary_gap=False,
            ),
        )
    )


#--------------------------合并------------------------------------------------
    grid = (
        Grid()
        .add(sd, grid_opts=opts.GridOpts(pos_bottom="25%"))#pos_left="60%"
        .add(bar, grid_opts=opts.GridOpts(pos_bottom="85%", pos_right="80%"))
        .add(int_line, grid_opts=opts.GridOpts(pos_top="75%"))
    )
    time_sd.add(grid, time_point=fea)

#--------------------------加入时间线------------------------------------------------
time_sd.add_schema(
    is_auto_play=False,#自启动
    play_interval=1000,
    width=1550,
    pos_left='10%',
    symbol_size=12,
    label_opts=opts.series_options.LabelOpts(
        # is_show=True,
        interval=0,
        font_size=11,
        # font_weight='bold'
        ),
)
page.add(time_sd)

In [75]:
#特征相关性热力图
def get_cor(feadata,fea_list):
    cor=feadata.iloc[:,:].corr()
    hatdata=[]
    for a in range(len(fea_list)):
        fea=fea_list[a]
        col=cor[fea]
        for b in range(len(fea_list)):
            fea=fea_list[b]
            y=col[fea]
            hatdata.append([b,a,round(abs(y),2)])
    return hatdata


keys=list(cross_dict.keys())
values=list(cross_dict.values())

die_fea={}
for i in range(len(keys)):
    for fea in values[i][1]:
        try:
            die_fea[fea].append(keys[i])
        except:
            die_fea[fea]=[keys[i]]

keys=list(die_fea.keys())
values=list(die_fea.values())

andnums={}
for a in range(len(keys)):
    aset=set(values[a])
    for b in range(len(keys)):
        bset=set(values[b])
        andset=aset & bset #两个特征越界人员的交集
#         andnum=len(andset)/min([len(aset),len(bset)])
        andnum=len(andset)
        try:
            andnums[keys[a]].append(andnum)
        except:
            andnums[keys[a]]=[andnum]
andnums_cor=pd.DataFrame(andnums,index=keys)

value=[]
for a in range(len(keys)):
    a_fea=keys[a]
    col=andnums_cor[a_fea]

    for b in range(len(keys)):
        b_fea=keys[b]
        y=col[b_fea]
        y=y/col[a_fea]
        value.append([b,a,round(abs(y),2)])

hat = (
    HeatMap(init_opts=opts.InitOpts(
                                    width='2000px',
                                    height='800px',
                                    chart_id='hat',
                                    theme=ThemeType.WESTEROS
                                    ))
    .add_xaxis(fea_list)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="特征相关性热力图"),
        visualmap_opts=opts.VisualMapOpts(
            min_=0, max_=1, is_calculable=True, orient="horizontal"
            , pos_right="15%",pos_top='0'
        ),
        legend_opts=opts.LegendOpts(selected_mode='single'),
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-45)),
    )
)

hat.add_yaxis(
        "特征相关性",
        fea_list,
        get_cor(np_fea,fea_list),
        label_opts=opts.LabelOpts(is_show=True, position="inside"),
    )

hat.add_yaxis(
        "越界重复度",
        fea_list,
        value,
        label_opts=opts.LabelOpts(is_show=True, position="inside"),
    )

page.add(hat)

##### 保存图表

In [76]:
print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))
print('💞💞 ' + tit + ' 绘图完毕')
page.page_title=tit
page.render("E:/desktop/chart/[对比]"+tit+".html")

Page.save_resize_html("E:/desktop/chart/[对比]"+tit+".html",
                      cfg_file="./chart_config (21).json",
                      dest="E:/desktop/chart/[对比]"+tit+".html")

os.system("E:/desktop/chart/[对比]"+tit+".html")

2022-11-26 11:14:15
💞💞 ★阴性整合+纯冰毒-重复 绘图完毕


0